In [1]:
import pandas as pd

In [2]:
data = pd.read_excel('./Network_LA/uc6_locals_junc_pair_v1.xlsx')
print (data.head(10))

   LinkID  JID1  JID2         StName
0       1   185   285  Henry Mayo Dr
1       2   285   305  Henry Mayo Dr
2       3   186   286  Henry Mayo Dr
3       4   286   306  Henry Mayo Dr
4       5   285   286  Henry Mayo Dr
5       6   944  1384   Lincoln Blvd
6       7  1384  1398   Lincoln Blvd
7       8  1384  1401   Lincoln Blvd
8       9   939   942   Lincoln Blvd
9      10   942   944   Lincoln Blvd


In [117]:
testing_data = []
for LinkID,JID1,JID2,StName in zip(data['LinkID'],data['JID1'],data['JID2'],data['StName']):
    if "Pioneer Blvd" in StName:
        testing_data.append([LinkID,JID1,JID2,StName])
dic = {}
street_id = {}
testing_data = sorted(testing_data,key=lambda x:x[1])
for row in testing_data:
    if row[1] not in dic:
        dic[row[1]] = [row[2]]
    elif row[2] not in dic[row[1]]:
        dic[row[1]].append(row[2])
    
    if row[2] not in dic:
        dic[row[2]] = [row[1]]
    elif row[1] not in dic[row[2]]:
        dic[row[2]].append(row[1])
    smaller = row[1]
    bigger = row[2]
    if smaller > bigger:
        smaller,bigger = bigger,smaller
    street_id[(smaller,bigger)] = row[0]
    
    #print (row)
multiple_road = {} 
for num in dic:
    if len(set(dic[num])) >=3:
        multiple_road[num] = dic[num]
#print (multiple_road)
print ("end reading data")

end reading data


In [119]:
#seen recorded sorted edge
#seen_pt record path of pt
#seen_edge record edges of path
def DFS(path,pre_element,next_element,seen,seen_pt,seen_edge):#seen saved the seen path

    if next_element in path:
        path = path+[next_element]   
        path = path[path.index(next_element):]
        cur_street = []
        for index in range(len(path)-1):
            smaller = path[index]
            bigger = path[index+1]
            if smaller > bigger:
                smaller,bigger = bigger,smaller
            cur_street.append(street_id[(smaller,bigger)])
        if sorted(cur_street) in seen: # make sure there is no redundant loop
            return None
        seen_edge.append(cur_street)
        seen.append(sorted(cur_street))
        seen_pt.append(path)
        return path 
    flag = None
    for new in dic[next_element]:
        if new != pre_element and new not in path[1:]:# make sure the circle should be from head to head
            flag = DFS(path+[next_element],next_element,new,seen,seen_pt,seen_edge)
            if flag != None:
                flag = True
                break
    if flag == None:
        return None
    return path
seen = [] # record the route edge
seen_edge = []
seen_pt = [] # record the  route point
for start_pos in multiple_road:
    #print ("round:",start_pos)
    for i in multiple_road[start_pos]:
        if DFS([start_pos],start_pos,i,seen,seen_pt,seen_edge) != None:
            break
print (len(seen))

51


In [131]:
union_edge = {}
merge_route = [False] * len(seen_pt)# record which road should be merge
merge_index = []
#print("multiple road dictionary",multiple_road)
print ("-------------")
for v in union_edge:
    if union_edge[v]>=2:
        print (v,union_edge[v])
for index,row in enumerate(seen):
    print (row)
    print (seen_pt[index])
    total = 0 
    for element in row:
        print ("or \"LinkID\" = ",element)
        union_edge[element] = union_edge.get(element,0)+1
    for pt in seen_pt[index]:
        if pt in multiple_road:
            total+=1
    if total -1 >2:
        merge_route[index] = True
    print ("TOTAL 3-WAY:",total-1)# -1 for it go back to original pt and we calculate start pos twice
    if total -1 >=3:
        merge_index.append(index)
seen = sorted([v for i,v in enumerate(seen) if i in merge_index],key = lambda x : len(x))
seen_pt = sorted([v for i,v in enumerate(seen_pt) if i in merge_index],key = lambda x : len(x))
seen_edge = sorted([v for i,v in enumerate(seen_edge) if i in merge_index],key = lambda x : len(x))


-------------
[4477, 4482, 4484, 4485, 4487, 4489, 4491, 4497, 4498, 4502, 4504, 4506, 4508, 4510, 4513, 4516, 4519, 4521, 4522, 4530, 4537, 4538, 4544, 4545, 4550, 4552, 4557]
[9088, 9087, 9085, 9079, 9078, 9077, 9076, 9075, 9073, 9074, 9072, 9035, 9036, 9051, 9056, 9055, 9080, 9081, 9082, 9083, 9084, 9086, 9089, 9090, 9093, 9094, 9095, 9088]
or "LinkID" =  4477
or "LinkID" =  4482
or "LinkID" =  4484
or "LinkID" =  4485
or "LinkID" =  4487
or "LinkID" =  4489
or "LinkID" =  4491
or "LinkID" =  4497
or "LinkID" =  4498
or "LinkID" =  4502
or "LinkID" =  4504
or "LinkID" =  4506
or "LinkID" =  4508
or "LinkID" =  4510
or "LinkID" =  4513
or "LinkID" =  4516
or "LinkID" =  4519
or "LinkID" =  4521
or "LinkID" =  4522
or "LinkID" =  4530
or "LinkID" =  4537
or "LinkID" =  4538
or "LinkID" =  4544
or "LinkID" =  4545
or "LinkID" =  4550
or "LinkID" =  4552
or "LinkID" =  4557
TOTAL 3-WAY: 27
[4477, 4482, 4485, 4487, 4489, 4491, 4498, 4502, 4504, 4508, 4510, 4513, 4519, 4522, 4529, 4538, 4

In [134]:
dif_group = 0
for i,val in enumerate(seen_pt):
    flag = False
    for j in range(i+1,len(seen_pt)):
        if all(n in seen_pt[j] for n in val):
            flag = True
            break
    if flag == False:
        dif_group+=1
        print (sorted(val))
'''
for pt_bucket in seen_pt[:-1]:
    if any(n not in seen_pt[-1] for n in pt_bucket):
        print (sorted(pt_bucket))
        dif_group +=1
'''
print ("seen ",seen_pt[-1])
print ("different group",dif_group)

[9025, 9034, 9034, 9041, 9050]
[9032, 9046, 9046, 9047, 9054]
[9017, 9019, 9020, 9027, 9027, 9031, 9037, 9038, 9043]
[8982, 8985, 8987, 8990, 8993, 9021, 9092, 9092, 9099, 9100]
[9035, 9036, 9051, 9055, 9056, 9072, 9073, 9074, 9075, 9076, 9077, 9078, 9079, 9080, 9081, 9082, 9083, 9084, 9085, 9086, 9087, 9088, 9089, 9090, 9091, 9091, 9093, 9094, 9095]
seen  [9091, 9088, 9087, 9085, 9079, 9078, 9077, 9076, 9075, 9073, 9074, 9072, 9035, 9036, 9051, 9056, 9055, 9080, 9081, 9082, 9083, 9084, 9086, 9089, 9090, 9093, 9094, 9095, 9091]
different group 5


In [87]:
delete_edge = []
for v in union_edge:
    if union_edge[v] >=2 and v not in seen[-1]:
        delete_edge.append(v)
print (delete_edge)

or "LinkID" =  4480
or "LinkID" =  4481
or "LinkID" =  4486
or "LinkID" =  4492
or "LinkID" =  4493
or "LinkID" =  4494
or "LinkID" =  4503
or "LinkID" =  4505
or "LinkID" =  4507
or "LinkID" =  4509
or "LinkID" =  4511
or "LinkID" =  4512
or "LinkID" =  4514
or "LinkID" =  4515
or "LinkID" =  4517
or "LinkID" =  4518
or "LinkID" =  4523
or "LinkID" =  4524
or "LinkID" =  4525
or "LinkID" =  4526
or "LinkID" =  4528
or "LinkID" =  4529
or "LinkID" =  4531
or "LinkID" =  4534
or "LinkID" =  4535
or "LinkID" =  4536
or "LinkID" =  4539
or "LinkID" =  4540
or "LinkID" =  4541
or "LinkID" =  4542
or "LinkID" =  4546
or "LinkID" =  4547
or "LinkID" =  4548
or "LinkID" =  4549
or "LinkID" =  4551
or "LinkID" =  4553
or "LinkID" =  4554
or "LinkID" =  4555
or "LinkID" =  4556
or "LinkID" =  4557
or "LinkID" =  4558
or "LinkID" =  4559
or "LinkID" =  4561
or "LinkID" =  4562
or "LinkID" =  4564
or "LinkID" =  4479
[4480, 4481, 4486, 4492, 4493, 4494, 4503, 4505, 4507, 4509, 4511, 4512, 4514, 4

In [30]:
route_pt = seen_pt[-1][:-1]
route_edge = seen_edge[-1]
for pt_id in route_pt:
    print ("or \"id\" = ",pt_id)
print (route_pt)
print (route_edge)

or "id" =  9091
or "id" =  9088
or "id" =  9087
or "id" =  9085
or "id" =  9079
or "id" =  9078
or "id" =  9077
or "id" =  9076
or "id" =  9075
or "id" =  9073
or "id" =  9074
or "id" =  9072
or "id" =  9035
or "id" =  9036
or "id" =  9051
or "id" =  9056
or "id" =  9055
or "id" =  9080
or "id" =  9081
or "id" =  9082
or "id" =  9083
or "id" =  9084
or "id" =  9086
or "id" =  9089
or "id" =  9090
or "id" =  9093
or "id" =  9094
or "id" =  9095
[9091, 9088, 9087, 9085, 9079, 9078, 9077, 9076, 9075, 9073, 9074, 9072, 9035, 9036, 9051, 9056, 9055, 9080, 9081, 9082, 9083, 9084, 9086, 9089, 9090, 9093, 9094, 9095]
[4520, 4530, 4521, 4497, 4537, 4508, 4522, 4550, 4502, 4489, 4510, 4498, 4519, 4485, 4477, 4513, 4491, 4487, 4545, 4552, 4538, 4482, 4504, 4544, 4516, 4484, 4506, 4533]


'\nmin_num  = min(route_pt)\nmax_num = max(route_pt)\nstart = route_pt.index(min_num)\nend = route_pt.index(max_num)\nstart,end = min(start,end),max(start,end)\nif end-start > len(route_pt):\n    route_pt =route_pt[start:end+1]\n    route_edge= route_edge[start:end]\nelse:\n    route_pt =route_pt[end:] + route_pt[:start+1]\n    route_edge= route_edge[end:] + route_edge[:start]\nfor edge_id in route_edge:\n    print ("or "LinkID" = ",edge_id)\nfor pt_id in route_pt:\n    print ("or "id" = ",pt_id)\n'

In [84]:
import numpy as np 
print (route_pt)

def read_pos(file_name,route_pt,route_edge): # read the position for the circle and find out the direction of the circle
    print ("-----------start_read_pos_of_all_pt-----------------")
    print (route_pt)
    data_pos = pd.read_excel(file_name)
    X_pos = []
    Y_pos = []
    for pt in route_pt:
        X_pos.append(data_pos.ix[pt-1,1]) # id 1 is first row
        Y_pos.append(data_pos.ix[pt-1,2])
    print ("std of x",np.std(X_pos))
    print ("std of y",np.std(Y_pos))
    N_S_Flag = False
    if np.std(Y_pos) > np.std(X_pos):
        N_S_Flag = True
    return read_critical_pt(N_S_Flag,X_pos,Y_pos,route_pt,route_edge)
def read_critical_pt(N_S_Flag,X_pos,Y_pos,route_pt,route_edge): #Use the direction to get the critical pt pair <northest,southest> or <eastest,westesst>
    print ("------------start_get_the_critical_pt---------------")
    print (route_pt)
    critical_pt1= critical_pt2 = -1 # store the boundary pt
    if N_S_Flag == True:
        print ("north_south_dir")
        critical_pt1 = Y_pos.index(max(Y_pos)) # find the northest pt index
        critical_pt1 = route_pt[critical_pt1] # get the id of the point
        critical_pt2 = Y_pos.index(min(Y_pos)) # find the southest pt index
        critical_pt2 = route_pt[critical_pt2] # get the id of the point
    else: # east west dir , haven't test it 
        critical_pt1 = X_pos.index(max(X_pos)) # find the eastest pt index
        critical_pt1 = route_pt[critical_pt1] # get the id of the point
        critical_pt2 = X_pos.index(min(X_pos)) # find the westest pt index
        critical_pt2 = route_pt[critical_pt2] # get the id of the point
        print ("east_west_dir")
    print ("critical pt=",critical_pt1,critical_pt2)
    return get_the_route_by_critical_pt(critical_pt1,critical_pt2,route_pt,route_edge)
def get_the_route_by_critical_pt(critical_pt1,critical_pt2,route_pt,route_edge):
    print ("------------start_get_the_critical_pt---------------")
    print (route_pt)
    
    start_index = route_pt.index(critical_pt1)
    end_index = route_pt.index(critical_pt2)
    print ("path index = ",start_index,end_index)
    start_index,end_index = min(start_index,end_index),max(start_index,end_index)
    if end_index - start_index+1 > int(len(route_pt)/2):
        print ("From start to end")
        route_pt =route_pt[start_index:end_index+1]
        route_edge= route_edge[start_index:end_index]
    else:
        route_pt =route_pt[end_index:] + route_pt[:start_index+1]
        route_edge= route_edge[end_index:] + route_edge[:start_index]
    for pt_id in route_pt:
        print ("or \"id\" = ",pt_id)
    for edge in route_edge:
        print ("or \"LinkID\" = ",edge)
    return route_edge,route_pt
filename = "./Network_LA/uc6_02_05_UTM_locals_ND_Junctions.xlsx"
print ("path =",read_pos(filename,route_pt,route_edge))


[9091, 9088, 9087, 9085, 9079, 9078, 9077, 9076, 9075, 9073, 9074, 9072, 9035, 9036, 9051, 9056, 9055, 9080, 9081, 9082, 9083, 9084, 9086, 9089, 9090, 9093, 9094, 9095]
-----------start_read_pos_of_all_pt-----------------
[9091, 9088, 9087, 9085, 9079, 9078, 9077, 9076, 9075, 9073, 9074, 9072, 9035, 9036, 9051, 9056, 9055, 9080, 9081, 9082, 9083, 9084, 9086, 9089, 9090, 9093, 9094, 9095]
std of x 37.4040428176
std of y 997.0067724
------------start_get_the_critical_pt---------------
[9091, 9088, 9087, 9085, 9079, 9078, 9077, 9076, 9075, 9073, 9074, 9072, 9035, 9036, 9051, 9056, 9055, 9080, 9081, 9082, 9083, 9084, 9086, 9089, 9090, 9093, 9094, 9095]
north_south_dir
critical pt= 9091 9051
------------start_get_the_critical_pt---------------
[9091, 9088, 9087, 9085, 9079, 9078, 9077, 9076, 9075, 9073, 9074, 9072, 9035, 9036, 9051, 9056, 9055, 9080, 9081, 9082, 9083, 9084, 9086, 9089, 9090, 9093, 9094, 9095]
path index =  0 14
From start to end
or "id" =  9091
or "id" =  9088
or "id" =  90

In [114]:
a = [8982, 8985, 8987, 8990, 8993, 9021, 9092, 9092, 9099, 9100]
for pt_id in a:
    print ("or \"id\" = ",pt_id)

or "id" =  8982
or "id" =  8985
or "id" =  8987
or "id" =  8990
or "id" =  8993
or "id" =  9021
or "id" =  9092
or "id" =  9092
or "id" =  9099
or "id" =  9100
